# Titanic Classification (XGBoost)

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV

### Setup

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
def wrangle(df):
    df["Male"] = df["Sex"].map(lambda x: x == "male")
    df["Pronoun"] = df["Name"].map(lambda x: x.split(", ")[1].split(".")[0]).astype("category")
    df["Surname"] = df["Name"].map(lambda x: x.split(",")[0]).astype("category")
    df["Brackets"] = df["Name"].map(lambda x: "(" in x)
    df["Prefix"] = df["Ticket"].map(lambda x: x.split(" ")[0].split("/")[0].replace(".", "") if " " in x else "").astype("category")
    df["Cabin"] = df["Cabin"].fillna("00").astype("category")
    df["Embarked"] = df["Embarked"].astype("category")
    df["Letter"] = df["Cabin"].fillna("00").map(lambda x: x[0]).astype("category")
    df["Embarked"] = df["Embarked"].astype("category")
    df = df.drop(["PassengerId", "Name", "Ticket", "Sex", "Cabin"], axis=1)
    return df

In [5]:
def encode(X_train, X_test, cat_cols):
    encoder = OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore')
    encoder.fit(X_train[cat_cols])
    X_train_encoded = encoder.transform(X_train[cat_cols])
    X_train_final = pd.concat([X_train.drop(cat_cols, axis=1), pd.DataFrame(X_train_encoded, index=X_train.index)], axis=1)
    X_test_encoded = encoder.transform(X_test[cat_cols])
    X_test_final = pd.concat([X_test.drop(cat_cols, axis=1), pd.DataFrame(X_test_encoded, index=X_test.index)], axis=1)
    return X_train_final, X_test_final

In [6]:
new_train = wrangle(train)

In [7]:
new_train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Male,Pronoun,Surname,Brackets,Prefix,Letter
0,0,3,22.0,1,0,7.2500,S,True,Mr,Braund,False,A,0
1,1,1,38.0,1,0,71.2833,C,False,Mrs,Cumings,True,PC,C
2,1,3,26.0,0,0,7.9250,S,False,Miss,Heikkinen,False,STON,0
3,1,1,35.0,1,0,53.1000,S,False,Mrs,Futrelle,True,,C
4,0,3,35.0,0,0,8.0500,S,True,Mr,Allen,False,,0


### Train test split on labelled data

In [8]:
# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    new_train.drop('Survived', axis=1),
    new_train['Survived'],
    test_size=0.2,
    random_state=42
)

In [9]:
cat_cols = ["Embarked", "Pronoun", "Surname", "Prefix", "Letter"]
X_train_final, X_test_final = encode(X_train, X_test, cat_cols)

/usr/local/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
X_train_final

,Pclass,Age,SibSp,Parch,Fare,Male,Brackets,0,1,2,...,592,593,594,595,596,597,598,599,600,601
331,1,45.5,0,0,28.5000,True,False,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
733,2,23.0,0,0,13.0000,True,False,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
382,3,32.0,0,0,7.9250,True,False,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
704,3,26.0,1,0,7.8542,True,False,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
813,3,6.0,4,2,31.2750,False,False,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,3,21.0,0,0,7.6500,False,False,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
270,1,NaN,0,0,31.0000,True,False,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
860,3,41.0,2,0,14.1083,True,False,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
435,1,14.0,1,2,120.0000,False,False,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
xgb_classifier = xgb.XGBClassifier()
xgb_classifier.fit(X_train_final, y_train)
y_pred = xgb_classifier.predict(X_test_final)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.7877094972067039


### Hyperparameter tuning

In [12]:
param_grid = {
    'max_depth': [2, 4, 6, 8],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200, 400],
    'gamma': [0, 0.1],
    'colsample_bytree': [0.6, 0.8, 1]
}

xgb_classifier = xgb.XGBClassifier()


grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_final, y_train)

print('Best hyperparameters:', grid_search.best_params_)

Best hyperparameters: {'colsample_bytree': 1, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 400}


In [13]:
best_xgb_classifier = grid_search.best_estimator_
y_pred = best_xgb_classifier.predict(X_test_final)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy with best hyperparameters:', accuracy)


Accuracy with best hyperparameters: 0.8268156424581006


### Train on the full dataset and predict test set survivors

In [14]:
new_test = wrangle(test)

In [15]:
X_train_final, X_test_final = encode(new_train.drop("Survived", axis=1), new_test, cat_cols)
y_train = new_train["Survived"]

/usr/local/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [16]:
xgb_classifier = xgb.XGBClassifier()


grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_final, y_train)

print('Best hyperparameters:', grid_search.best_params_)

Best hyperparameters: {'colsample_bytree': 0.6, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100}


In [17]:
best_xgb_classifier = grid_search.best_estimator_
y_pred = best_xgb_classifier.predict(X_test_final)

In [18]:
prediction = pd.DataFrame()
prediction["PassengerId"] = test["PassengerId"]
prediction["Survived"] = y_pred
prediction

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [19]:
prediction.to_csv("prediction.csv", index=False)